In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Import ImageDataGenerator

from tensorflow.keras.callbacks import LearningRateScheduler

2024-03-27 19:20:50.179091: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-27 19:20:50.179226: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-27 19:20:50.373899: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
train_data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_data = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [4]:
X_train = train_data.drop(columns=['label']).values
y_train = train_data['label'].values

In [5]:
X_train = X_train.reshape(-1, 28, 28, 1) / 255.0
X_test = test_data.values.reshape(-1, 28, 28, 1) / 255.0


In [6]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [7]:
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)
datagen.fit(X_train)

In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [9]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 30:
        lr *= 0.5
    elif epoch > 20:
        lr *= 0.75
    return lr

In [10]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
callbacks = [
    LearningRateScheduler(lr_schedule)
]

In [13]:
history = model.fit(datagen.flow(X_train, y_train, batch_size=64), epochs=40, validation_data=(X_val, y_val), callbacks=callbacks)

Epoch 1/40


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


591/591 ━━━━━━━━━━━━━━━━━━━━ 52s 84ms/step - accuracy: 0.6883 - loss: 0.9218 - val_accuracy: 0.9836 - val_loss: 0.0584 - learning_rate: 0.0010
Epoch 2/40
591/591 ━━━━━━━━━━━━━━━━━━━━ 50s 84ms/step - accuracy: 0.9474 - loss: 0.1755 - val_accuracy: 0.9879 - val_loss: 0.0409 - learning_rate: 0.0010
Epoch 3/40
591/591 ━━━━━━━━━━━━━━━━━━━━ 51s 85ms/step - accuracy: 0.9638 - loss: 0.1197 - val_accuracy: 0.9864 - val_loss: 0.0389 - learning_rate: 0.0010
Epoch 4/40
591/591 ━━━━━━━━━━━━━━━━━━━━ 81s 84ms/step - accuracy: 0.9678 - loss: 0.1065 - val_accuracy: 0.9919 - val_loss: 0.0245 - learning_rate: 0.0010
Epoch 5/40
591/591 ━━━━━━━━━━━━━━━━━━━━ 82s 83ms/step - accuracy: 0.9746 - loss: 0.0844 - val_accuracy: 0.9914 - val_loss: 0.0295 - learning_rate: 0.0010
Epoch 6/40
591/591 ━━━━━━━━━━━━━━━━━━━━ 82s 84ms/step - accuracy: 0.9768 - loss: 0.0792 - val_accuracy: 0.9924 - val_loss: 0.0321 - learning_rate: 0.0010
Epoch 7/40
591/591 ━━━━━━━━━━━━━━━━━━━━ 81s 82ms/step - accuracy: 0.9790 - loss: 0.0671